In [1]:
%cd ..

/Users/ishamrashik/Desktop/machine-learning-and-ai-projects/community-mcp-session/aws_community_day/youtube-sentiment-insights


In [ ]:
import mlflow
import os 

# Set AWS credentials (replace with your actual credentials)
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION')

# Then set MLflow tracking URI
mlflow.set_tracking_uri("http://3.29.137.20:5000")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

<Experiment: artifact_location='s3://mlflow-artifacts-5fb77f1c/346044369985827915', creation_time=1760222953342, experiment_id='346044369985827915', last_update_time=1760222953342, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [7]:
from utilities import load_data, INTERIM_DATA_PATH

In [8]:
train_df_path = os.path.join(INTERIM_DATA_PATH, "train_processed.csv")
val_df_path = os.path.join(INTERIM_DATA_PATH, "val_processed.csv")
test_df_path = os.path.join(INTERIM_DATA_PATH, "test_processed.csv")

train_df = load_data(train_df_path)
val_df = load_data(val_df_path)
test_df = load_data(test_df_path)

2025-10-12 02:51:49,991 - model_building - DEBUG - Data loaded from data/interim/train_processed.csv
2025-10-12 02:51:49,995 - model_building - DEBUG - Data loaded from data/interim/val_processed.csv
2025-10-12 02:51:50,000 - model_building - DEBUG - Data loaded from data/interim/test_processed.csv


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [10]:

# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    # Step 1: Vectorize the text comments (Bag of Words)
    X_train_text = vectorizer.fit_transform(train_df['clean_comment']).toarray()
    X_val_text = vectorizer.transform(val_df['clean_comment']).toarray()

    # Step 2: Extract numerical features
    # Define the numerical feature columns to include
    numerical_features = ['word_count', 'num_stop_words', 'num_chars', 'num_chars_cleaned']

    X_train_numerical = train_df[numerical_features].values
    X_val_numerical = val_df[numerical_features].values

    # Step 3: Combine text features with numerical features
    import numpy as np

    X_train = np.hstack([X_train_text, X_train_numerical])
    X_val = np.hstack([X_val_text, X_val_numerical])

    # Target variable
    y_train = train_df['category']
    y_val = val_df['category']

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_val)

        # Log accuracy
        accuracy = accuracy_score(y_val, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_val, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_val, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


2025/10/12 02:54:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 02:54:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://3.29.137.20:5000/#/experiments/346044369985827915/runs/fe79d8f3efe74d78810ba4f1a9230318
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/346044369985827915


/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

🏃 View run TF-IDF_(1, 1)_RandomForest at: http://3.29.137.20:5000/#/experiments/346044369985827915/runs/d946f34a67bc433cb52c9be73ec650dd
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/346044369985827915


2025/10/12 02:55:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 02:55:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://3.29.137.20:5000/#/experiments/346044369985827915/runs/25a3766ea2f74e12bee9874664106422
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/346044369985827915


2025/10/12 02:55:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/12 02:55:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://3.29.137.20:5000/#/experiments/346044369985827915/runs/c19ad2a9815d4e54b5adf8985393a75c
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/346044369985827915


/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/mlops_aws_community/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

🏃 View run TF-IDF_(1, 3)_RandomForest at: http://3.29.137.20:5000/#/experiments/346044369985827915/runs/1162f0523d6243ed8ee8a24ac3c49505
🧪 View experiment at: http://3.29.137.20:5000/#/experiments/346044369985827915


In [12]:
import mlflow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load your experiment data
experiment = mlflow.get_experiment_by_name("Exp 2 - BoW vs Tfidf")